# Cafe Sales Analytics Projects

## Objectives
fill the missing values in the cleaned datasets Cafe Sales.

### SETUP for Filling values

In [6]:
import pandas as pd
import numpy as np

In [7]:
#import CSV
data_path = "../data/processed/cleaned_cafe_sales_without_filling.csv"
try:
    df = pd.read_csv(data_path)
except FileNotFoundError:
    print("ERROR : File Not Found")

In [8]:
#make a copy to fill
fill_df = df.copy()
pd.set_option("display.max_row", None)

In [9]:
fill_df.columns

Index(['Transaction ID', 'Item', 'Quantity', 'Price Per Unit', 'Total Spent',
       'Payment Method', 'Location', 'Transaction Date'],
      dtype='object')

### Fill missing values in 'Item'